# Covert Timing Channels

### Overview

We will denote $m$ to be the size of the secret message. Also let $B$ denote the maximum number of packets that can be buffered at anytime. The issue of having a large $B$ is that packets that are at the end of the buffer  must wait for all the packets that in the front to be transmitted. This may cause unacceptable delays for the packets. The value of $B$ will be determined by the overt application that ALice is using. For example, for real-time application (such as Augmented Reality/Virtual Reality applications) $B$ will be small whereas for Email application $B$ can be large.   In this study we will consider different values of $m$ and $B$ with $m \geq B$ and Alice, Eve and Bob know the value of $B$ since they all know what is the overt application Alice is using. 

Given $m$ and $B$ what should be Alice's strategy be to tranmit the secret message? There are two important constraints: 

1. Alice cannot buffer more than $B$ packets because of the reason above.
2. Once she starts to transmit the secret message she cannot stop (because she has no way of letting Bob know). She must try to compelete sending the entire secret message.

Alice follows the following strategy. Before starting to transmit the secret message she buffers $i$ ($0 \geq i \leq B$) packets and then starts to release the packets to transmit the secret message. In order to determine what should $i$ be we need to discuss two system states that we need to worry about - buffer overflow and buffer underflow. Let's understand what these are, why these can occur and what are the factors that determine when they will occur. 


1. **Buffer Overflow**: This happens when the buffer already has $B$ packets and another packet arrives from the application. Recall we have said that the number of packets in the buffer cannot exceed $B$. To build  intuition as to when this happens let's consider a specific scenario. Suppose we have set $B = 10$, $m = 32$ and $i=8$. Since $i = 8$,  Alice will first buffer 8 packets. As soon as the 8th packet arrives from the application, she will start transmitting packets with inter-packet delays that encode the secret message.  While she release the packet from the buffer, new packets may arrive from the application that will be appended to the buffer. So the number of packets in the buffer will keep changing - decrease when a packet is transmitted and increase when a packet is generated by the application. Suppose at some time there 7 packets in the buffer and before the next packet is to be transmitted 4 packets arrive in quick succession from the application. This will cause the number of packets in the buffer to go beyond 10 and that will be a violation of policy at most 10 packets can be buffered. Essentially, if packets arrive faster than they are transmitted out, there will be buffer overflow. If $i$ is set close to $B$ there is likely to be an overflow. 


2. **Buffer Underflow**: This happens when a packet must be transmitted (to encode a bit of the secret message) but there are no packets in the buffer. Recall the constraint that once Alice starts to transmit the secret message she cannot stop. Hence, if packets arrive slower than they are transmitted there is likely  be buffer underflow. If $i$ is set close to 0, there is likely to be an underflow. 


**If either of the above two cases (buffer overflow and buffer underflow) arise during the process of sending the secret message we will consider that to be a failure of the covert channel. 


There are many factors that will determine if the covert channel will go into overflow and underflow. These include $m$, $B$, $i$, distribution of the inter-packet delay of the overt application, and the encoding scheme. In this part of the project we will write a simulation to study some of the above factors on the success of transmitting a secret message.  The overall sender-side system is shown in the Figure below. 


<figure> 
    <img src="Figures/ctc-implementation.jpg" width="600" height="400">
    <figcaption align = "center"><b>The overall system diagram of the source and the covert sender. </b></figcaption>
</figure>



### Assumptions

  1. We will consider that the source generates packet following well-known IPD distributions. Specifically, we will consider two cases a) Exponential and b) Uniform. The sender (Alice) also knows this distribution and follows it to inject the delay between the packets to embed the secret message. It is important to note that the source and the sender are independent. Hence, even though they follow the same distribution,  the sequence of delays generated by the source will be different from the sequence of delays generated by the sender.
  
  2. To embed a 0, the sender generates a delay between the minimum value (min) and the median. To embed a 1 the sender generates a delay between the median value and the maximum value (max). Note that  for the Uniform distribution the min, max and median are easy to determine. For the Exponential distribution min is 0, the max is $\infty$. What is the median value of an Exponential distribution with rate parameter $\lambda$ pkts/sec?
  
  3. The secret message is a randomly generated sequence of 1s and 0s of size $m$ bits and is given. We will consider two values $m=16, 32$.
  
  4. The sender has a buffer of size $B$ and initially the sender buffers $i$ packets before starting to send the secret message.
  
  
  
### Project Steps 

  1. For  buffer size $B=20$ we want to find out the probability of overflow  and  underflow, when the IPD follows the Exponential with $\lambda =1$ pkts/sec and $i=2, 6, 10, 14, 18$. Use message size $m = 16, 32$ bits. Tabulate the results. Remember that to determine the probability you need to run multiple (say 500) experiments for each parameter, i.e., for $B = 20, m = 16, i = 2$ run 500 experiments  and determine the probability of overflow and underflow. Similarly for other values of $i$ and $m$. The max value of an Exponentia.l distribution is $\infty$. For this we can limit the max value to say 5 ~secs$. 
  
  2. For  buffer size $B=20$ we want to find out the probability of overflow  and  underflow, when the IPD follows the Uniform distribution in the range (0,1)  and $i=2, 6, 10, 14, 18$. Use message size $m = 16, 32$ bits.  Tabulate the results. Remember that to determine the probability you need to run multiple (say 500) experiments for each parameter, i.e., for $B = 20, m = 16, i = 2$ run 500 experiments  and determine the probability of overflow and underflow. Similarly for other values of $i$ and $m$.


  3. Propose methods to deal with buffer overflow and underflow.



### 1. Exponentially distributed packet delays, with rate=1 packets/sec

In [5]:
import random
import math

n = 1000                   #number of experiments for each set of parameters
B = 20                     #size of buffer
m = [16,32]                #sizes of secret message
i = [2,6,10,14,18]         #number of packets buffered before sender begins transmission

Min = 0                    #min of Exponential distribution with rate=1
Med = math.log(2)          #(ln(2)) median of Exponential distribution with rate=1
Max = 5                    #upper limit, max of exponential distributions is infinity

underflow_prob = []
overflow_prob = []
success_prob = []

for j in m:                       #iterate through different message sized
    for k in i:                   #iterate through different buffer size before transmission values
        #keep track of number of buffer overflows and underlows
        overflow_count = 0
        underflow_count = 0
        success_count = 0
        for l in range(n):        #run n experiments for each pair of m and i values
            CB = k                                    #current buffer size
            time = 0
            message = []
            packet_times = [0]
            success = 1
            for a in range(j):
                message.append(random.randint(0, 1))               #randomized secret message
                delay = random.expovariate(1)
                while delay > 5:
                    delay = random.expovariate(1)
                packet_times.append(delay+packet_times[a])         #generated packet times
            
            for b in message:                                      #iterate through message bits
                if CB <= 0:                                        #check for underflow
                    underflow_count = underflow_count + 1
                    success = 0
                    break
                if b == 0:                                         #generate packet_delay
                    packet_delay = random.uniform(Min,Med)
                else:
                    packet_delay = random.uniform(Med,Max)
                time = time + packet_delay                         #update time
                CB = CB - 1
                for c in range(len(packet_times)):                 #add generated packets to buffer
                    if packet_times[c] <= time:
                        CB = CB + 1
                        packet_times[c] = 9999
                if CB >= B:                                        #check for overflow
                    overflow_count = overflow_count + 1
                    success = 0
                    break
            if success == 1:
                success_count = success_count + 1                  #increment success count if no overflow/undeflow
        
        #calculate buffer underflow/overflow probabilities for each set of m and i values
        underflow_prob.append(underflow_count/n)            
        overflow_prob.append(overflow_count/n)
        #calculate success probability
        success_prob.append(success_count/n)

print("When generated packet delays follow an exponential distribution with rate of 1 packet/sec:")
print("(The resulting probabilities are in the order: i = 2, 6, 10, 14, 18)")
print("\nBuffer Underflow")
print("16-bit message:",underflow_prob[0],underflow_prob[1],underflow_prob[2],underflow_prob[3],underflow_prob[4],sep="  ")
print("32-bit message:",underflow_prob[5],underflow_prob[6],underflow_prob[7],underflow_prob[8],underflow_prob[9],sep="  ")
print("\nBuffer Overflow")
print("16-bit message:",overflow_prob[0],overflow_prob[1],overflow_prob[2],overflow_prob[3],overflow_prob[4],sep="  ")
print("32-bit message:",overflow_prob[5],overflow_prob[6],overflow_prob[7],overflow_prob[8],overflow_prob[9],sep="  ")
print("\nSuccess")
print("16-bit message:",success_prob[0],success_prob[1],success_prob[2],success_prob[3],success_prob[4],sep="  ")
print("32-bit message:",success_prob[5],success_prob[6],success_prob[7],success_prob[8],success_prob[9],sep="  ")

When generated packet delays follow an exponential distribution with rate of 1 packet/sec:
(The resulting probabilities are in the order: i = 2, 6, 10, 14, 18)

Buffer Underflow
16-bit message:  0.24  0.019  0.001  0.0  0.0
32-bit message:  0.23  0.039  0.002  0.0  0.0

Buffer Overflow
16-bit message:  0.0  0.007  0.243  0.721  0.961
32-bit message:  0.21  0.499  0.794  0.939  0.995

Success
16-bit message:  0.76  0.974  0.756  0.279  0.039
32-bit message:  0.56  0.462  0.204  0.061  0.005


### 2. Uniformly distributed packet delays, with range(0,1)

In [7]:
import random

n = 1000                   #number of experiments for each set of parameters
B = 20                     #size of buffer
m = [16,32]                #sizes of secret message
i = [2,6,10,14,18]         #number of packets buffered before sender begins transmission

Min = 0                    #min of Uniform distribution with a=0 and b=1
Med = 0.5                  #median of Uniform distribution with a=0 and b=1
Max = 1                    #max of Uniform distribution with a=0 and b=1

underflow_prob = []
overflow_prob = []
success_prob = []

for j in m:                       #iterate through different message sized
    for k in i:                   #iterate through different buffer size before transmission values
        #keep track of number of buffer overflows and underlows
        overflow_count = 0
        underflow_count = 0
        success_count = 0
        for l in range(n):        #run n experiments for each pair of m and i values
            CB = k                                    #current buffer size
            time = 0
            message = []
            packet_times = [0]
            success = 1
            for a in range(j):
                message.append(random.randint(0, 1))                       #randomized secret message
                packet_times.append(random.uniform(0,1)+packet_times[a])   #generated packet times
            
            for b in message:                                      #iterate through message bits
                if CB <= 0:                                        #check for underflow
                    underflow_count = underflow_count + 1
                    success = 0
                    break
                if b == 0:                                         #generate packet_delay
                    packet_delay = random.uniform(Min,Med)
                else:
                    packet_delay = random.uniform(Med,Max)
                time = time + packet_delay                         #update time
                CB = CB - 1
                for c in range(len(packet_times)):                 #add generated packets to buffer
                    if packet_times[c] <= time:
                        CB = CB + 1
                        packet_times[c] = 9999
                if CB >= B:                                        #check for overflow
                    overflow_count = overflow_count + 1
                    success = 0
                    break
            if success == 1:
                success_count = success_count + 1                  #increment success count if no overflow/undeflow
        
        #calculate buffer underflow/overflow probabilities for each set of m and i values
        underflow_prob.append(underflow_count/n)            
        overflow_prob.append(overflow_count/n)
        #calculate success probability
        success_prob.append(success_count/n)

print("When generated packet delays follow a uniform distribution with range(0,1):")
print("(The resulting probabilities are in the order: i = 2, 6, 10, 14, 18)")
print("\nBuffer Underflow")
print("16-bit message:",underflow_prob[0],underflow_prob[1],underflow_prob[2],underflow_prob[3],underflow_prob[4],sep="  ")
print("32-bit message:",underflow_prob[5],underflow_prob[6],underflow_prob[7],underflow_prob[8],underflow_prob[9],sep="  ")
print("\nBuffer Overflow")
print("16-bit message:",overflow_prob[0],overflow_prob[1],overflow_prob[2],overflow_prob[3],overflow_prob[4],sep="  ")
print("32-bit message:",overflow_prob[5],overflow_prob[6],overflow_prob[7],overflow_prob[8],overflow_prob[9],sep="  ")
print("\nSuccess")
print("16-bit message:",success_prob[0],success_prob[1],success_prob[2],success_prob[3],success_prob[4],sep="  ")
print("32-bit message:",success_prob[5],success_prob[6],success_prob[7],success_prob[8],success_prob[9],sep="  ")

When generated packet delays follow a uniform distribution with range(0,1):
(The resulting probabilities are in the order: i = 2, 6, 10, 14, 18)

Buffer Underflow
16-bit message:  0.413  0.02  0.0  0.0  0.0
32-bit message:  0.589  0.143  0.013  0.001  0.0

Buffer Overflow
16-bit message:  0.0  0.0  0.001  0.076  0.648
32-bit message:  0.0  0.0  0.037  0.207  0.732

Success
16-bit message:  0.587  0.98  0.999  0.924  0.352
32-bit message:  0.411  0.857  0.95  0.792  0.268


### 3. Dealing with buffer underflow and overflow

In [38]:
#      When the sender starts the transmission of packets at a large buffer size, such as 18 when maximum
# buffer size is 20, the rate of undeflow was the lowest, but the rate of overflow was the highest.
# Similarly, when the sender starts the transmission of packets at a small buffer size, such as 2, the
# rate of overflow was the lowest, but the rate of underflow was the highest.
# 
#      As illustrated by the calculated probabilities, the best buffer size at which to start sending packets
# is the middle ground, something like 10. This compromise has a low probability of both underflow and overflow.
# Attempting to lower the probability of one raises the probability of the other.
# 
#      Overflow was an especially large issue when packets were generated along an exponential distribution.
# In cases like this, where overflow was more prevalent than underflow, we can lower the threshold value
# to increase the average rate that packets are transmitted by the sender. If the opposite was the case,
# and undeflow was a larger issue, then we can increase the threshold value to let packets stay in the
# buffer longer on average before transmission.